<a href="https://colab.research.google.com/github/joekelly211/masfi/blob/main/6_scenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports, directories and global functions

In [ ]:
# Define base directory
base_dir = "/gdrive/Shareddrives/masfi"

# Mount Google Drive and set base directory
from google.colab import drive
import os
import sys
drive.mount('/gdrive', force_remount=True)
_path_to_add = os.path.realpath(base_dir)
if _path_to_add not in sys.path:
    sys.path.append(_path_to_add)

In [ ]:
# Capture outputs
%%capture
# Installs and upgrades
!pip install astropy
!pip install geopandas
!pip install rasterio
!pip install xgboost --upgrade
!apt-get install -y gdal-bin

In [ ]:
# Reload imports, replacing those in the cache
%reload_ext autoreload
%autoreload 2
# Imports
from astropy.convolution import convolve, Gaussian2DKernel
import geopandas as gpd
from google.colab import runtime
import itertools
import json
from os import makedirs
from os.path import join, exists
from osgeo import gdal
import ipywidgets as widgets
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
import psutil
import rasterio
from rasterio.features import rasterize
import re
from scipy import ndimage
import shutil
from shutil import copyfile
import tensorflow as tf
import xgboost as xgb

In [ ]:
# Define directories
areas_dir = join(base_dir, "1_areas")
polygons_dir = join(areas_dir, "polygons")

predictor_dir = join(base_dir, "3_predictors")
predictor_resampled_dir = join(predictor_dir, "resampled")
predictor_final_dir = join(predictor_dir, "final")
models_dir = join(base_dir, "5_models")
scenarios_dir = join(base_dir, "6_scenarios")
masks_dir = join(scenarios_dir, "scenario_masks")

# Create directories
makedirs(scenarios_dir, exist_ok=True)
makedirs(masks_dir, exist_ok=True)

In [ ]:
# Global function: export an array as a .tif
template_tif_path = join(areas_dir, "template.tif")
nodatavalue = -1111111
compress = True
def export_array_as_tif(input_array, output_tif, template=template_tif_path, nodatavalue=nodatavalue, compress=compress):
  template = gdal.Open(template)
  template_band = template.GetRasterBand(1)
  template_dimensions, template_projection = template.GetGeoTransform(), template.GetProjection()
  if compress: driver = gdal.GetDriverByName("GTiff").Create(output_tif, template_band.XSize, template_band.YSize, bands=1, eType=gdal.GDT_Float32,
                                                options=["COMPRESS=DEFLATE","PREDICTOR=2","ZLEVEL=9"])
  if compress == False: driver = gdal.GetDriverByName("GTiff").Create(output_tif, template_band.XSize, template_band.YSize, bands=1, eType=gdal.GDT_Float32)
  driver.GetRasterBand(1).WriteArray(input_array)
  driver.GetRasterBand(1).SetNoDataValue(nodatavalue)
  driver.SetGeoTransform(template_dimensions)
  driver.SetProjection(template_projection)

# Burn a polygon to raster
def burn_polygon_to_raster(raster, polygon, fixed=True, fixed_value=1, column_name=None, all_touched=True):
  with rasterio.open(raster, 'r+') as src:
      array = src.read(1)
      transform = src.transform
      gdf = gpd.read_file(polygon)
      for geom in gdf.geometry:
          if not fixed and column_name == None:
              column_name = gdf.columns[0]
          if not fixed: burn_value = gdf.loc[gdf.geometry == geom, column_name].values[0]
          else: burn_value = fixed_value
          rasterize([(geom, burn_value)], out=array, transform=transform,
              all_touched=all_touched, dtype=src.meta['dtype'], out_shape=src.shape)
      src.write(array, 1)

# Select model and scenario area

In [ ]:
# Select a model
model_exists = False
for subdir, dirs, files in os.walk(models_dir):
  for file in files:
    if file == 'model.json':
      print(f'selected_model = "{subdir.split(f"{models_dir}/",1)[1]}"')
      model_exists = True
if not model_exists:
  print("No model exists.")

In [ ]:
selected_model = "agbd_240926_030225"
categorise_variate = False # If the variate was categorised in 5_models

# Define model directories
selected_model_dir = join(models_dir,selected_model)
selected_model_json = join(selected_model_dir, "model.json")
selected_model_descr_dir = join(selected_model_dir, "model_description.json")
selected_model_dataset_path = join(selected_model_dir, f"{selected_model}.pkl")
selected_model_dataset = pd.read_pickle(selected_model_dataset_path)

# Read description for model dataset attributes
with open(join(selected_model_dir,"model_dataset_description.json")) as model_dataset_description_json:
  model_dataset_description = json.load(model_dataset_description_json)
model_dataset_name = model_dataset_description["model_dataset_name"]
number_of_columns = model_dataset_description["number_of_columns"]
number_of_rows = model_dataset_description["number_of_rows"]
id_column = model_dataset_description["id_column"]
selected_variate = model_dataset_description["selected_variate"]
uncertainty = model_dataset_description["uncertainty"]
covariates_renamed = model_dataset_description["covariates_renamed"]
covariates_categorised = model_dataset_description["covariates_categorised"]
selected_predictors = model_dataset_description["selected_predictors"] + model_dataset_description["covariates_renamed"]
categorical_columns = model_dataset_description["categorical_columns"]
descriptive_parameters = model_dataset_description["descriptive_parameters"]
filter_parameter = model_dataset_description["filter_parameter"]
filter_values_to_include = model_dataset_description["filter_values_to_include"]
sample_imported_dataset = model_dataset_description["sample_imported_dataset"]
sample_imported_dataset_by_percent = model_dataset_description["sample_imported_dataset_by_percent"]
sample_imported_dataset_value = model_dataset_description["sample_imported_dataset_value"]

covariates = [covariate[4:] for covariate in covariates_renamed]

# Create scenarios model directory
scenarios_model_dir = join(scenarios_dir, selected_model)
makedirs(scenarios_model_dir, exist_ok=True)

# Copy model_dataset_description.json
with open(join(scenarios_model_dir, "model_dataset_description.json"), "w") as file:
  file.write(json.dumps(model_dataset_description))

# Select a scenario area
scenario_area_exists = False
for subdir in os.listdir(scenarios_model_dir):
  if not subdir.endswith('.csv') and not subdir.endswith('.json'):
    print(f'selected_scenario_area = "{subdir}"')
    scenario_area_exists = True
if not scenario_area_exists:
  print(f"Create a scenario area directory in {scenarios_model_dir}")

In [ ]:
selected_scenario_area = "tekai"

# Define scenario area directory
scenario_area_dir = join(scenarios_model_dir,selected_scenario_area)
makedirs(scenario_area_dir, exist_ok=True)

# Create subdirectories
predictors_dir = join(scenario_area_dir, "predictors")
tile_templates_dir = join(scenario_area_dir, 'tile_templates')
tile_predictors_dir = join(scenario_area_dir, "tile_predictors")
tile_predictor_stacks_dir = join(scenario_area_dir, "tile_predictor_stacks")
tile_prediction_cache_dir = join(scenario_area_dir,"tile_prediction_cache")
scenario_predictions_unmasked_dir = join(scenario_area_dir,"scenario_predictions_unmasked")
scenario_predictions_dir = join(scenario_area_dir, "scenario_predictions")
scenario_diff_dir = join(scenario_area_dir, "scenario_difference")

makedirs(predictors_dir, exist_ok=True)
makedirs(tile_templates_dir, exist_ok=True)
makedirs(tile_predictors_dir, exist_ok=True)
makedirs(tile_predictor_stacks_dir, exist_ok=True)
makedirs(tile_prediction_cache_dir, exist_ok=True)
makedirs(scenario_predictions_unmasked_dir, exist_ok=True)
makedirs(scenario_predictions_dir, exist_ok=True)
makedirs(scenario_diff_dir, exist_ok=True)

# Copy predictors from the final predictors directory
for predictor in os.listdir(predictor_final_dir):
  if predictor not in os.listdir(predictors_dir):
    predictor_original_path = join(predictor_final_dir, predictor)
    predictor_copy_path = join(predictors_dir, predictor)
    copyfile(predictor_original_path, predictor_copy_path)
print(f"All predictors present in the following directory have already been copied over: {predictor_final_dir}")

# Define scenarios

In [ ]:
# Yearly scenarios may include years after the model scenario if predictor data is available later than GEDI data.
# Date of the prediction is ~December 31st, e.g. '2023' is 31/12/2023, requiring predictors up to 2023.

model_scenario_override = None # set if cannot be automatically determined from model predictors

yearly_predictors = ["forest_with_edge_effects", "disturbance_with_edge_effects"]

# Remove the 'pre_' prefix from each predictor
model_predictors = sorted([predictor[4:] for predictor in selected_predictors])

# Create a list of predictor years from the model's predictors
model_predictor_years = []
for predictor in model_predictors:
  for yearly_predictor in yearly_predictors:
    if yearly_predictor in predictor:
      model_predictor_years.append(int(predictor[-4:]))

# Determine the model scenario from the maximum year
# Values from the most recent predictor year (e.g. 2022) will be applied to the second most recent (e.g. 2021) as a proxy at the predictor stack stage
if model_scenario_override != None: model_scenario = model_scenario_override
else: model_scenario = max(model_predictor_years)
model_scenario_filename = f"{model_scenario}.csv"
model_scenario_dir = join(scenarios_model_dir,model_scenario_filename)
print(f"The maximum year used in the model is {model_scenario}, which has been created as the first scenario.\n")
print(f"The {model_scenario} scenario predictor list has been saved to:\n {model_scenario_dir}\n")
print(f"Ensure all predictors in this list have been copied to:\n{predictors_dir}\n")

# Save the model scenario predictors as a .csv
pd.DataFrame(model_predictors).to_csv(model_scenario_dir, index=False)

# Determine available predictor years
final_predictor_years = []
for final_predictor in os.listdir(predictor_final_dir):
  if final_predictor.endswith('.tif') and final_predictor[-9] == '_':
    try: final_predictor_years.append(int(final_predictor[-8:-4]))
    except: continue

# Find the first and last predictor years
first_predictor_year = min(final_predictor_years)
last_predictor_year = max(final_predictor_years)
additional_predictor_years = last_predictor_year - model_scenario
print(f"The first available predictor year is {first_predictor_year} and the last is {last_predictor_year}.\n")

# Calculate the range of scenario years and minimum scenario year
model_scenario_year_range = max(model_predictor_years) - (min(model_predictor_years))
minimum_yearly_scenario = first_predictor_year + model_scenario_year_range
print(f"The earliest scenario year that can be predicted is {minimum_yearly_scenario}.")
print(f"The latest scenario year that can be predicted is {last_predictor_year}.")
print(f"This is based on the number of yearly predictors used to train the model and the total availability of predictors.")

In [ ]:
# Select constant predictors which are the same in every scenario, e.g. topography
print("constant_predictors = [")
for predictor in model_predictors:
  print(f'  "{predictor}",')
print("]")

In [ ]:
constant_predictors = [
  "coast_proximity_km",
  "latitude",
  "longitude",
  "topo_cor_smooth_aspect_cosine",
  "topo_cor_smooth_aspect_sine",
  "topo_cor_smooth_circular_variance_aspect_03",
  "topo_cor_smooth_circular_variance_aspect_07",
  "topo_cor_smooth_circular_variance_aspect_11",
  "topo_cor_smooth_deviation_mean_elevation_03",
  "topo_cor_smooth_deviation_mean_elevation_07",
  "topo_cor_smooth_deviation_mean_elevation_11",
  "topo_cor_smooth_eastness",
  "topo_cor_smooth_elevation",
  "topo_cor_smooth_northness",
  "topo_cor_smooth_profile_curvature",
  "topo_cor_smooth_roughness_03",
  "topo_cor_smooth_roughness_07",
  "topo_cor_smooth_roughness_11",
  "topo_cor_smooth_slope",
  "topo_cor_smooth_stream_power_index_log10",
  "topo_cor_smooth_surface_area_ratio",
  "topo_cor_smooth_tangential_curvature",
  "topo_cor_smooth_topographic_position_index_03",
  "topo_cor_smooth_topographic_position_index_07",
  "topo_cor_smooth_topographic_position_index_11",
  "topo_cor_smooth_topographic_ruggedness_index",
  "topo_cor_smooth_topographic_wetness_index",
  "topo_cor_unsmooth_aspect_cosine",
  "topo_cor_unsmooth_aspect_sine",
  "topo_cor_unsmooth_circular_variance_aspect_03",
  "topo_cor_unsmooth_circular_variance_aspect_07",
  "topo_cor_unsmooth_circular_variance_aspect_11",
  "topo_cor_unsmooth_deviation_mean_elevation_03",
  "topo_cor_unsmooth_deviation_mean_elevation_07",
  "topo_cor_unsmooth_deviation_mean_elevation_11",
  "topo_cor_unsmooth_eastness",
  "topo_cor_unsmooth_elevation",
  "topo_cor_unsmooth_northness",
  "topo_cor_unsmooth_profile_curvature",
  "topo_cor_unsmooth_roughness_03",
  "topo_cor_unsmooth_roughness_07",
  "topo_cor_unsmooth_roughness_11",
  "topo_cor_unsmooth_slope",
  "topo_cor_unsmooth_stream_power_index_log10",
  "topo_cor_unsmooth_surface_area_ratio",
  "topo_cor_unsmooth_tangential_curvature",
  "topo_cor_unsmooth_topographic_position_index_03",
  "topo_cor_unsmooth_topographic_position_index_07",
  "topo_cor_unsmooth_topographic_position_index_11",
  "topo_cor_unsmooth_topographic_ruggedness_index",
  "topo_cor_unsmooth_topographic_wetness_index",
]

In [ ]:
# Note that areas that were forested in a historic yearly scenario but were reservoirs
# or flooded at the time topographic predictor data was collected (~2014 for GLO-30 DEM)
# will be predicted based on a flat topography at the elevation of the water's surface.

# Set scenario predictors as all non-constant predictors
scenario_predictors = sorted(list(set(model_predictors) - set(constant_predictors)))

# Create predictor lists for all possible yearly scenarios
for yearly_scenario in range(minimum_yearly_scenario, last_predictor_year +1):
  year_difference = model_scenario - yearly_scenario
  yearly_scenario_predictors = []
  for scenario_predictor in scenario_predictors:
    try:
      year_change = int(scenario_predictor[-4:]) - year_difference
      yearly_scenario_predictor = scenario_predictor[:-4] + str(year_change)
      yearly_scenario_predictors.append(yearly_scenario_predictor)
    except: yearly_scenario_predictors.append(scenario_predictor)
  # Compile yearly predictors and save as a .csv
  yearly_scenario_predictors = sorted(yearly_scenario_predictors + constant_predictors)
  yearly_scenario_filename = f"{yearly_scenario}.csv"
  yearly_scenario_dir = join(scenarios_model_dir,yearly_scenario_filename)
  pd.DataFrame(yearly_scenario_predictors).to_csv(yearly_scenario_dir, index=False)

# Open the most recent yearly scenario predictor list
most_recent_scenario_csv = join(scenarios_model_dir,f"{last_predictor_year}.csv")
most_recent_scenario_predictors = pd.Series.tolist(pd.read_csv(most_recent_scenario_csv).iloc[:,0])

print(f"Lists of predictors for all possible yearly scenarios have been exported to {scenarios_model_dir}/.")
print(f"Ensure all predictors in these lists have been copied to:\n{predictors_dir}\n")

# Create a 'no disturbance' predictor for alternate scenarios.
# Assumes the minimum possible value is present in the first scenario year.
minimum_disturbance_name = f"disturbance_with_edge_effects_0000"
minimum_disturbance_path = join(predictors_dir, f"{minimum_disturbance_name}.tif")
if not exists(minimum_disturbance_path):
  example_disturbance = join(predictors_dir, f"disturbance_with_edge_effects_{first_predictor_year}.tif")
  example_disturbance_array = gdal.Open(example_disturbance).ReadAsArray()
  minimum_disturbance_value = example_disturbance_array.min()
  minimum_disturbance_array = np.where(example_disturbance_array, minimum_disturbance_value, minimum_disturbance_value)
  export_array_as_tif(minimum_disturbance_array, minimum_disturbance_path, template = example_disturbance)
  print(f"The minimum disturbance value is {minimum_disturbance_value}\n,")
  print(f"which has been used to create the 'minimum disturbance' predictor {minimum_disturbance_name}.")
else: print(f"The minimum disturbance predictor {minimum_disturbance_name} already exists.")

In [ ]:
# These alternate scenarios remove degradation or deforestation and degradation
# prior to the most recent yearly scenario (e.g. 2023)

# 'No degradation' scenarios remove disturbance since the indicated year
define_nodeg_scenarios = True

no_degradation_since = [
    first_predictor_year,
    # 2000,
]

for year in no_degradation_since:
  assert year >= first_predictor_year, "Years in 'no_degradation_since' must be set to or after the first predictor year."


# 'No degradation and no deforestation' scenarios remove disturbance
# and add the forest lost since the indicated year
define_nodeg_nodef_scenarios = True

no_deforestation_or_degradation_since = [
    first_predictor_year,
    # 2000,
]

for year in no_deforestation_or_degradation_since:
  assert year >= first_predictor_year, "Years in 'no_deforestation_or_degradation_since' must be set to or after the first predictor year."

# Create a predictor list for 'no degradation' scenarios
if define_nodeg_scenarios:
  for year in no_degradation_since:
    nodeg_predictors = []
    for scenario_predictor in most_recent_scenario_predictors:
      if "disturbance_with_edge_effects" in scenario_predictor:
        scenario_predictor_year = int(scenario_predictor[-4:])
        if scenario_predictor_year > year: # Only remove disturbance from years specified
          nodeg_predictors.append(minimum_disturbance_name)
        else: nodeg_predictors.append(scenario_predictor)
      else: nodeg_predictors.append(scenario_predictor)
    nodeg_predictors = sorted(nodeg_predictors) # Sort to ensure order is maintained
    nodeg_scenario_filename = f"{last_predictor_year}_nodeg_since_{year}.csv"
    nodeg_scenario_path = join(scenarios_model_dir, nodeg_scenario_filename)
    pd.DataFrame(nodeg_predictors).to_csv(nodeg_scenario_path, index=False)
    print(f"Predictor list for a scenario without degradation between")
    print(f"{last_predictor_year} and {year} exported to {nodeg_scenario_filename}.\n")
else: print("The 'no degradation' scenarios are not enabled.\n")

# Create a predictor list for 'no Landsat deforestation and no Landsat degradation'
if define_nodeg_nodef_scenarios:
  for year in no_deforestation_or_degradation_since:
    nodeg_nodef_predictors = []
    for scenario_predictor in most_recent_scenario_predictors:
      if "disturbance_with_edge_effects" in scenario_predictor:
        scenario_predictor_year = int(scenario_predictor[-4:])
        if scenario_predictor_year > year: # Only remove disturbance from years specified
          nodeg_nodef_predictors.append(minimum_disturbance_name)
        else: nodeg_nodef_predictors.append(scenario_predictor)
      elif "forest_with_edge_effects" in scenario_predictor:
        scenario_predictor_year = int(scenario_predictor[-4:])
        if scenario_predictor_year > year: # Replace with specified forest cover year
          nodeg_nodef_predictors.append(f"forest_with_edge_effects_{year}")
        else: nodeg_nodef_predictors.append(scenario_predictor)
      else: nodeg_nodef_predictors.append(scenario_predictor)
    nodeg_nodef_predictors = sorted(nodeg_nodef_predictors) # Sort to ensure order is maintained
    nodeg_nodef_scenario_filename = f"{last_predictor_year}_nodeg_nodef_since_{year}.csv"
    nodeg_nodef_scenario_path = join(scenarios_model_dir, nodeg_nodef_scenario_filename)
    pd.DataFrame(nodeg_nodef_predictors).to_csv(nodeg_nodef_scenario_path, index=False)
    print(f"Predictor list for a scenario without degradation or deforestation between")
    print(f"{last_predictor_year} and {year} exported to {nodeg_nodef_scenario_filename}.\n")

else: print("The 'no degradation and no deforestation' scenarios are not enabled.")

In [ ]:
# These alternate scenarios simulate old-growth forest using a proxy area specified by the user.
# Forest extent (i.e. 'no deforestation') can be set from yearly predictor, or the entire land area.

define_oldgrowth_scenarios = True
oldgrowth_yearly_scenarios = [
    first_predictor_year,
    # 2020,
    last_predictor_year
]

for year in oldgrowth_yearly_scenarios:
  assert year in final_predictor_years, "Years in 'oldgrowth_yearly_scenarios' must be available in the final yearly predictors."

simulate_oldgrowth_all_land = True
oldgrowth_all_land_name = f"{first_predictor_year}_oldgrowth_all_land"

# The predictor that best indicates oldgrowth to the model, e.g. certain protected areas
# This will be modified to cover the entire scenario area for the old-growth scenarios
oldgrowth_predictor = 'lu_pa_taman_negara_krau_with_edge_effects'

# Some predictors may confound the old-growth proxy, e.g. protected areas that are not known to be old-growth
# These will be removed for the old-growth scenarios
oldgrowth_redundant_predictors = [
'lu_ais_with_edge_effects',
'lu_berkelah_jerantut_with_edge_effects',
'lu_berkelah_kuantan_with_edge_effects',
'lu_berkelah_temerloh_with_edge_effects',
'lu_remen_chereh_with_edge_effects',
'lu_tekai_tembeling_with_edge_effects',
'lu_tekam_with_edge_effects',
'lu_yong_lipis_with_edge_effects',
'lu_yong_with_edge_effects',
]

if define_oldgrowth_scenarios:
  # Expand the oldgrowth predictor to the entire scenario area
  oldgrowth_predictor_all_dir = join(predictors_dir, f"{oldgrowth_predictor}_all.tif")
  if not exists(oldgrowth_predictor_all_dir):
    oldgrowth_predictor_dir = join(predictors_dir, f"{oldgrowth_predictor}.tif")
    oldgrowth_predictor_array = gdal.Open(oldgrowth_predictor_dir).ReadAsArray()
    oldgrowth_predictor_max_value = oldgrowth_predictor_array.max()
    print(f"The maximum value for the oldgrowth predictor '{oldgrowth_predictor}' is {oldgrowth_predictor_max_value}.")
    oldgrowth_predictor_all_array = np.where(oldgrowth_predictor_array, oldgrowth_predictor_max_value, oldgrowth_predictor_max_value)
    oldgrowth_predictor_all_dir = join(predictors_dir, f"{oldgrowth_predictor}_all.tif")
    export_array_as_tif(oldgrowth_predictor_all_array, oldgrowth_predictor_all_dir, template = oldgrowth_predictor_dir)
    print(f"The oldgrowth proxy {oldgrowth_predictor} has been expanded to the entire scenario area")
    print(f"And exported to {oldgrowth_predictor_all_dir}")
  else: print(f"The oldgrowth predictor '{oldgrowth_predictor}_all.tif' already exists.\n")

  # Remove the redundant predictors from the oldgrowth scenario area
  for redundant_predictor in oldgrowth_redundant_predictors:
    redundant_predictor_none_dir = join(predictors_dir, f"{redundant_predictor}_none.tif")
    if not exists(redundant_predictor_none_dir):
      redundant_predictor_dir = join(predictors_dir, f"{redundant_predictor}.tif")
      redundant_predictor_array = gdal.Open(redundant_predictor_dir).ReadAsArray()
      redundant_predictor_min_value = redundant_predictor_array.min()
      print(f"The minimum value for the redundant predictor {redundant_predictor} is {redundant_predictor_min_value}.")
      redundant_predictor_none_array = np.where(redundant_predictor_array, redundant_predictor_min_value, redundant_predictor_min_value)
      export_array_as_tif(redundant_predictor_none_array, redundant_predictor_none_dir, redundant_predictor_dir)
      print(f"The oldgrowth redundant predictor {oldgrowth_predictor} has been removed from the entire scenario area")
      print(f"And exported to {oldgrowth_predictor_all_dir}.")
    else: print(f"The oldgrowth redundant predictor '{redundant_predictor}_none.tif' already exists.\n")

  # Generate 'oldgrowth' scenarios for each specified year (i.e. forest extent in that year)
  for year in oldgrowth_yearly_scenarios:
    oldgrowth_predictors = []
    for scenario_predictor in most_recent_scenario_predictors:
      if "disturbance_with_edge_effects" in scenario_predictor:
        oldgrowth_predictors.append(minimum_disturbance_name)
      elif "forest_with_edge_effects" in scenario_predictor:
        scenario_predictor_year = int(scenario_predictor[-4:])
        if scenario_predictor_year > year: # Replace with specified forest cover year
          oldgrowth_predictors.append(f"forest_with_edge_effects_{year}")
        else: oldgrowth_predictors.append(scenario_predictor)
      elif scenario_predictor == oldgrowth_predictor:
        oldgrowth_predictors.append(f"{oldgrowth_predictor}_all")
      elif scenario_predictor in oldgrowth_redundant_predictors:
        oldgrowth_predictors.append(f"{redundant_predictor}_none")
      else: oldgrowth_predictors.append(scenario_predictor)
    oldgrowth_predictors = sorted(oldgrowth_predictors)
    oldgrowth_filename = f"{year}_oldgrowth.csv"
    oldgrowth_dir = join(scenarios_model_dir, oldgrowth_filename)
    pd.DataFrame(oldgrowth_predictors).to_csv(oldgrowth_dir, index=False)
    print(f"Predictor list for a scenario where all forest in {first_predictor_year} was old-growth")
    print(f"has been exported to {oldgrowth_filename}.\n")


  if simulate_oldgrowth_all_land:
    # Based on the first TMF AnnualChanges land coverage
    # Create a forest predictor all land that exists in the first predictor year (e.g. 1990)
    forest_all_land_name = f"forest_with_edge_effects_{first_predictor_year}_all_land"
    forest_all_land_path = join(predictors_dir, f"{forest_all_land_name}.tif")
    first_annual_changes_filename = f"tmf_AnnualChanges_Dec{first_predictor_year}.tif"
    first_annual_changes_path = join(predictor_resampled_dir, first_annual_changes_filename)
    if not exists(forest_all_land_path):
      if exists(first_annual_changes_path):
        first_annual_changes_array = gdal.Open(first_annual_changes_path).ReadAsArray()
        # Convert all water values to 'nodata' and non-water values to '1'
        forest_all_land_array = np.where(first_annual_changes_array == 5, nodatavalue, 1)
        # Set smoothing kernel and precision
        kernel, precision = Gaussian2DKernel(x_stddev=1, y_stddev=1), 2
        # Reclassify for binary differentiation after proximity conversion
        differentiator_array = forest_all_land_array.copy()
        differentiator_array[differentiator_array == 1] = 10
        # Positive proximity
        positive_distances = ndimage.distance_transform_edt(forest_all_land_array == 0) # target pixels
        positive_proximity_array = np.where(positive_distances > 2, 0, positive_distances) # max distance 2
        # Negative proximity
        negative_distances = ndimage.distance_transform_edt(forest_all_land_array == 1) # target pixels
        negative_proximity_array = np.where(negative_distances > 2, 0, negative_distances) # max distance 2
        # Sum proximities and differentiator
        pixel_prox_summed =  differentiator_array + positive_proximity_array + negative_proximity_array
        # Reclassify for better semantic understanding of pixel proximity
        pixel_prox_reclassed = pixel_prox_summed.copy()
        pixel_prox_reclass_table = [(0, 0, -4), (1, 1, -1), (1.4, 1.5, -2), (2, 2, -3), (10, 10, 3), (11, 11, 0), (11.4, 11.5, 1), (12, 12, 2)]
        for min_value, max_value, new_value in pixel_prox_reclass_table:
          pixel_prox_reclassed[(pixel_prox_reclassed >= min_value) & (pixel_prox_reclassed <= max_value)] = new_value
        # Smooth binary array using 2D convolution
        binary_smoothed = convolve(forest_all_land_array, kernel, boundary='extend')
        # Sum pixel proximity and smoothed binary array
        edge_effects_array = np.round(pixel_prox_reclassed + binary_smoothed, precision)
        # Export forest edge effects predictor
        export_array_as_tif(edge_effects_array, forest_all_land_path)
        print(f"{forest_all_land_name} has been created and saved to\n{predictors_dir}\n")
      else: print(f"The first TMF annual changes year raster needed for comrec is not in the indicated directory:\n{first_annual_changes_path}\n")
    print(f"{forest_all_land_name} already exists in\n{predictors_dir}\n")
    if exists(forest_all_land_path):
      oldgrowth_all_predictors = []
      for scenario_predictor in most_recent_scenario_predictors:
        if "disturbance_with_edge_effects" in scenario_predictor:
          oldgrowth_all_predictors.append(minimum_disturbance_name)
        elif "forest_with_edge_effects" in scenario_predictor:
          oldgrowth_all_predictors.append(forest_all_land_name)
        elif scenario_predictor == oldgrowth_predictor:
          oldgrowth_all_predictors.append(f"{scenario_predictor}_all")
        elif scenario_predictor in oldgrowth_redundant_predictors:
          oldgrowth_all_predictors.append(f"{scenario_predictor}_none")
        else: oldgrowth_all_predictors.append(scenario_predictor)
      oldgrowth_all_predictors = sorted(oldgrowth_all_predictors)
      oldgrowth_all_filename = f"{oldgrowth_all_land_name}.csv"
      oldgrowth_all_dir = join(scenarios_model_dir, oldgrowth_all_filename)
      pd.DataFrame(oldgrowth_all_predictors).to_csv(oldgrowth_all_dir, index=False)
      print(f"Predictor list for a scenario with old-growth covering everywhere that was land in {first_predictor_year}\n")
      print(f"has been exported to {oldgrowth_all_filename}.\n")

    # Create A forest mask for 'all land' oldgrowth
    # Also forests reservoirs since the first TMF annual changes year, though topography may be wrong
    if exists(join(scenarios_model_dir, f"{oldgrowth_all_land_name}.csv")):
      if exists(first_annual_changes_path):
        oldgrowth_all_mask_path = join(masks_dir, f"mask_forest_{first_predictor_year}_all_land.tif")
        if not exists(oldgrowth_all_mask_path):
          first_annual_changes_array = gdal.Open(first_annual_changes_path).ReadAsArray()
          # Convert all water values to 'nodata' and non-water values to '1'
          oldgrowth_all_mask_array = np.where(first_annual_changes_array == 5, nodatavalue, 1)
          export_array_as_tif(oldgrowth_all_mask_array, oldgrowth_all_mask_path)
          print(f"A mask for 'oldgrowth_all_land_name' has been created at\n{oldgrowth_all_mask_path}")
        else: print(f"A mask for 'oldgrowth_all_land_name' already exists at\n{oldgrowth_all_mask_path}")
      else: print(f"The {first_annual_changes_filename} raster needed to mask {oldgrowth_all_land_name} doesn't exist.")
    else: print(f"The scenario csv for {oldgrowth_all_land_name} doesn't exist.")

else: print("Old-growth scenarios are not enabled.")

# Predictor verification

In [ ]:
# Check that all predictors in all scenario csvs exist
scenario_csv_list = []
all_predictors_exist = True # Changes to false if predictor missing
for csv in os.listdir(scenarios_model_dir):
  if csv.endswith('.csv'):
    csv_dir = join(scenarios_model_dir, csv)
    csv_predictor_list = pd.Series.tolist(pd.read_csv(csv_dir).iloc[:,0])
    csv_predictor_dir_list = []
    for csv_predictor in csv_predictor_list:
      if csv_predictor not in covariates: csv_predictor_dir_list.append(f"{predictors_dir}/{csv_predictor}.tif")
    for predictor in csv_predictor_dir_list:
      if not exists(predictor):
        all_predictors_exist = False
        print(f"The following predictor is missing:\n{predictor}\n and is required for the scenario '{csv[:-4]}'")

if all_predictors_exist: print("All required predictors are present.")
print("Covariate predictors e.g. 'beam' and 'sensitivity' will be added at the prediction stage.")

In [ ]:
# Is the scenario area equal to the original template area?
original_template_area = True

# If not, create a new template for the scenario area and upload to:
# '6_scenarios/[model]/[scenario_area]/template.tif'
if original_template_area: scenario_template_dir = join(areas_dir, "template.tif")
else: scenario_template_dir = join(scenario_area_dir, "template.tif")
print(f"The following is being used as a template to verify scenario predictor dimensions and projections:\n{scenario_template_dir}")


scenario_template = gdal.Open(scenario_template_dir)
scenario_template_dimensions, scenario_template_projection = scenario_template.GetGeoTransform(), scenario_template.GetProjection()

In [ ]:
predictor_issue = False
for predictor in os.listdir(predictors_dir):
  if predictor.endswith('.tif'):
    predictor_dir = join(predictors_dir, predictor)
    predictor_open = gdal.Open(predictor_dir)
    predictor_dimensions, predictor_projection = predictor_open.GetGeoTransform(), predictor_open.GetProjection()
    if predictor_dimensions != scenario_template_dimensions:
      print(f"{predictor} dimensions:\n{predictor_dimensions}\ndo not match the scenario template dimensions:\n{scenario_template_dimensions}\n")
      predictor_issue = True
    if predictor_projection != scenario_template_projection:
      print(f"{predictor} projection:\n{predictor_projection}\ndoes not match the scenario template projection:\n{scenario_template_projection}\n\n")
      predictor_issue = True

if not predictor_issue: print(f"All predictors in the following directory have the correct dimensions and projection:\n{predictors_dir}")
else: print("Correct and / or resample the predictor(s).")

# Template tiles

In [ ]:
# Load the model scenario predictors for tile template creation
model_scenario_predictors = pd.Series.tolist(pd.read_csv(model_scenario_dir).iloc[:,0])
model_scenario_predictors_dirs = [predictors_dir + '/' + predictor + '.tif' for predictor in model_predictors]
# Create a template predictor array from the first predictor that isn't a covariate (these are created later)
template_base_path = next(r for r in model_scenario_predictors_dirs if all(c not in r for c in covariates))
template_base = gdal.Open(template_base_path)
template_base_array = template_base.ReadAsArray()
template_base_xsize, template_base_ysize = template_base.GetRasterBand(1).XSize, template_base.GetRasterBand(1).YSize
print(f"The template predictor is {template_base_xsize} x {template_base_ysize} pixels.")

# Check existing tile parameters
template_tile_list = []
for file in os.listdir(tile_templates_dir):
  if file.endswith('.tif') and file[:13] == 'template_tile':
    template_tile_list.append(file)
n_tiles_exist = len(template_tile_list)

if n_tiles_exist < 1: print("There are currently no template tiles. Run the next section.")
if n_tiles_exist >= 1:
  tile_size_y_rounded_exist = gdal.Open(join(tile_templates_dir,'template_tile_1.tif')).GetRasterBand(1).YSize
  tile_size_y_remainder_exist = gdal.Open(join(tile_templates_dir,f'template_tile_{n_tiles_exist}.tif')).GetRasterBand(1).YSize
  if n_tiles_exist == 1: print(f"There is a single 'tile' with a height of {tile_size_y_rounded_exist}."); tile_size_y_remainder_exist = 0
  else: print(f"There are {n_tiles_exist} template tiles, the first {n_tiles_exist-1} having a height of {tile_size_y_rounded_exist} pixels, the last {tile_size_y_remainder_exist} pixels.")

In [ ]:
# Large scenario areas and / or numbers of predictors may be too much for the available memory.
# This section defines how to split predictions into tiles that can then be merged.
override_n_tiles = False # Useful if the tile number has already been tested.
n_tiles_override = 1
tile_size_y_rounded_override = int(14910/n_tiles_override)

memory_utilisation = 0.8 # Set to 0.8 to ensure crashes are avoided

assert memory_utilisation > 0 and memory_utilisation <= 1, "Set memory_utilisation to a value between 0 and 1"

# Calculate total size of predictor stack
predictor_stack_size = template_base_array.size * len(model_scenario_predictors_dirs)

# Calculate memory and the number of tiles required
total_memory_needed = 64 / 8 * predictor_stack_size
print(f'RAM required for each prediction: ~{total_memory_needed/(1024**3):.3f} GB')
print(f'RAM currently available: {psutil.virtual_memory().free / (1024**3):.3f} GB')
n_tiles_temp = int(np.ceil(total_memory_needed / (psutil.virtual_memory().free * memory_utilisation)))

# Calculate template tile size (split on the y axis only)
tile_size_y_rounded = int(np.ceil(template_base_ysize/n_tiles_temp)) # Round the number of y pixels in each tile
tile_size_y_remainder = template_base_ysize%tile_size_y_rounded # Calculate the remainder for the last tile
n_tiles = max(1, len(range(0, template_base_ysize, tile_size_y_rounded))) # Update the number of tiles to include the remainder

if override_n_tiles:
  tile_size_y_rounded = tile_size_y_rounded_override
  tile_size_y_remainder = template_base_ysize%tile_size_y_rounded
  n_tiles = n_tiles_override
  print("n_tiles has been overridden.")

print(f'The prediction template should be divided into {n_tiles} tiles to avoid crashing.')

# Check if tiles need to be changed
if n_tiles == n_tiles_exist and tile_size_y_rounded == tile_size_y_rounded_exist and tile_size_y_remainder == tile_size_y_remainder_exist:
  print("No changes to existing tiles are required.")
else:
  # Clear all tile directories
  for tile in Path(tile_templates_dir).glob("**/*"):
    if tile.is_file(): tile.unlink()
  for tile in Path(tile_predictors_dir).glob("**/*"):
    if tile.is_file(): tile.unlink()
  for scenario_stack_dir in Path(tile_predictor_stacks_dir).glob("**/*"):
    shutil.rmtree(scenario_stack_dir)
  for tile in Path(tile_prediction_cache_dir).glob("**/*"):
    if tile.is_file(): tile.unlink()

  # Generate new tile templates based on available memory
  tile_number = 1
  for y_start in range(0, template_base_ysize, tile_size_y_rounded):
    if tile_size_y_remainder != 0 and tile_number == n_tiles: tile_size_y = tile_size_y_remainder
    else: tile_size_y = tile_size_y_rounded
    tiling_string = "gdal_translate -of GTIFF -srcwin " + str(0)+ ", " + str(y_start) + ", " + str(template_base_xsize) + ", " + str(tile_size_y) + " " + str(template_base_path) + " " + str(tile_templates_dir) + "/template_tile_" + str(tile_number) + ".tif"
    os.system(tiling_string)
    tile_number += 1

  print("Template tile creation complete.")

# Predictor tiles

In [ ]:
# Create predictor tiles.

# Check existing tile parameters
template_tile_list = []
for file in os.listdir(tile_templates_dir):
  if file.endswith('.tif') and file[:13] == 'template_tile':
    template_tile_list.append(file)
n_tiles = len(template_tile_list)

assert n_tiles > 0, "There are no template tiles. Run the template tiles section, even if only one is created."
tile_size_y_rounded = gdal.Open(join(tile_templates_dir,'template_tile_1.tif')).GetRasterBand(1).YSize
print(f"There are {n_tiles} template tiles.")

if n_tiles == 1: print("Predictor tile creation skipped. Predictor stack creation will use the original predictors.")
else:
  # Progress
  n_predictors = len(os.listdir(predictors_dir))
  predictor_progress_index, predictor_progress_label = 0, widgets.Label(value=f"Predictor progress: 0 / {n_predictors}")
  display(predictor_progress_label)
  tile_progress_index, tile_progress_label = 0, widgets.Label(value=f"Tile progress: 0 / {n_tiles}")
  display(tile_progress_label)

  # Loop through each predictor in the 6_scenarios predictors directory
  for predictor in os.listdir(predictors_dir):
    # Create list of tile directories
    predictor_dir = join(predictors_dir, predictor)
    predictor_array = gdal.Open(predictor_dir).ReadAsArray()
    # Split the predictor array into chunks based on tile size
    predictor_chunks = np.array_split(predictor_array, np.arange(tile_size_y_rounded, len(predictor_array), tile_size_y_rounded))
    tile_count = 1
    # Loop through tiles and export as .tif
    for tile in range(n_tiles):
      predictor_tile_filename = f"{predictor[:-4]}_{tile_count}.tif"
      # Check if tile already exists
      predictor_tile_exists = False
      for predictor_tile in os.listdir(tile_predictors_dir):
        if predictor_tile == predictor_tile_filename: predictor_tile_exists=True
        # If predictor tile does not exist:
      if predictor_tile_exists == False:
        template_tile_dir = join(tile_templates_dir, f"template_tile_{tile_count}.tif")
        export_array_as_tif(predictor_chunks[tile_count-1], join(tile_predictors_dir,predictor_tile_filename), template_tile_dir, compress = False)
        tile_count += 1
      # Update tile progress
      tile_progress_index += 1
      tile_progress_label.value = f"Tile progress: {tile_progress_index} / {n_tiles}"
    tile_progress_index = 0
    # Update predictor progress
    predictor_progress_index += 1
    predictor_progress_label.value = f"Predictor progress: {predictor_progress_index} / {n_predictors}"

# Predictor stacks

In [ ]:
# Create predictor stack arrays for each scenario

# Collect scenarios with .csv predictor lists
scenarios_list = []
for csv in os.listdir(scenarios_model_dir):
  if csv.endswith('.csv'):
    scenarios_list.append(csv[:-4])

# Select scenarios to generate tiled predictor stacks
print("scenarios_to_stack = [")
for scenario in sorted(scenarios_list):
  print(f'  "{scenario}",')
print("]")

In [ ]:
scenarios_to_stack = [
  "1990_oldgrowth",
  "1990_oldgrowth_all_land",
  # "2016",
  "2018",
  "2019",
  "2020",
  "2021",
  "2022",
  "2023",
  "2023_nodeg_nodef_since_1990",
  "2023_nodeg_since_1990",
  "2023_oldgrowth",
]

assert set(scenarios_to_stack).issubset(scenarios_list), "Not all selected scenarios exist."

# Check existing tile parameters
template_tile_list = []
for file in os.listdir(tile_templates_dir):
  if file.endswith('.tif') and file[:13] == 'template_tile':
    template_tile_list.append(file)
n_tiles = len(template_tile_list)

assert n_tiles > 0, "There are no template tiles. Run the template tiles section, even if only one is created."
print(f"There are {n_tiles} template tiles.")

# Progress
scenario_progress_index, scenario_progress_label = 0, widgets.Label(value=f"Scenario progress: 0 / {len(scenarios_to_stack)}")
display(scenario_progress_label)
stack_progress_index, stack_progress_label = 0, widgets.Label(value=f"Tiled predictor stack progress: 0 / {n_tiles}")
display(stack_progress_label)

# Loop through each scenario
for scenario in scenarios_to_stack:
    # Define directory and predictors
    scenario_predictor_stacks_dir = join(tile_predictor_stacks_dir, scenario)
    makedirs(scenario_predictor_stacks_dir, exist_ok=True)
    scenario_predictors_csv = join(scenarios_model_dir,f"{scenario}.csv")
    scenario_predictors = pd.Series.tolist(pd.read_csv(scenario_predictors_csv).iloc[:,0])
    # Set the number of stacks to the number of tiles
    if n_tiles == 0: n_stacks = 1
    else: n_stacks = n_tiles
    # Create a tile count to match the predictor stack
    tile_count = 1
    for tile in range(n_stacks):
      scenario_stack_filename = f"predictor_stack_{scenario}_{tile_count}.npy"
      # Check if predictor stack already exists
      predictor_stack_exists = False
      for predictor_stack in os.listdir(scenario_predictor_stacks_dir):
        if predictor_stack == scenario_stack_filename: predictor_stack_exists=True
      # If scenario prediction tile does not exist:
      if predictor_stack_exists == False:
        scenario_tile_stack_dir = join(scenario_predictor_stacks_dir, scenario_stack_filename)
        # Create predictor chunks (arrays) from tiles
        if n_stacks == 1: predictor_tiles_dirs = [f"{predictors_dir}/{predictor}.tif" for predictor in scenario_predictors]
        else: predictor_tiles_dirs = [f"{tile_predictors_dir}/{predictor}_{tile_count}.tif" for predictor in scenario_predictors]
        predictor_array_chunks = []
        for predictor in predictor_tiles_dirs:
          # Covariate raster will exist and should be ignored if prediction stage has already been attempted
          if predictor.split('/')[-1].split('.')[0] not in covariates and predictor.split('/')[-1].split('.')[0] not in [f"{cov}_{tile_count}" for cov in covariates]:
            predictor_array_chunk = gdal.Open(predictor).ReadAsArray()
            predictor_array_chunks.append(predictor_array_chunk)
        # Create a predictor stack from chunks
        predictor_stack = np.dstack(predictor_array_chunks)
        predictor_array_chunks = None # Flush chunks
        stack_height, stack_width, stack_n_predictors = predictor_stack.shape
        # Convert predictor stack to 2D numpy array with predictors as columns
        predictor_stack_reshaped = predictor_stack.reshape(stack_height * stack_width, stack_n_predictors)
        predictor_stack = None # Flush stack
        # Save as a numpy file
        np.save(scenario_tile_stack_dir, predictor_stack_reshaped)
        predictor_stack_reshaped = None # Flush reshaped stack
      # Update progress
      tile_count += 1
      stack_progress_index += 1
      stack_progress_label.value = f"Tiled predictor stack progress: {stack_progress_index} / {n_stacks}"
    # Reset tile progress
    stack_progress_index = 0
    scenario_progress_index += 1
    scenario_progress_label.value = f"Scenario progress: {scenario_progress_index} / {len(scenarios_to_stack)}"
print("\nPredictor stacks complete.")

# Predict scenarios

In [ ]:
# Collect available scenarios from the predictor stack tiles directory
scenario_stacks_list = []
for scenario in os.listdir(tile_predictor_stacks_dir):
    scenario_stacks_list.append(scenario)

# Select scenarios to predict
print("scenarios_to_predict = [")
for scenario in sorted(scenario_stacks_list):
  print(f'  "{scenario}",')
print("]")

In [ ]:
scenarios_to_predict = [
  "1990_oldgrowth",
  "1990_oldgrowth_all_land",
  # "2016",
  "2018",
  "2019",
  "2020",
  "2021",
  "2022",
  "2023",
  "2023_nodeg_nodef_since_1990",
  "2023_nodeg_since_1990",
  "2023_oldgrowth",
]

assert set(scenarios_to_predict).issubset(scenario_stacks_list), "Not all selected scenarios exist."

In [ ]:
add_covariates = True # Adds a selected covariate value as the predictor
sensitivity_value = 0.99
beam_value = 5
# 5 is the first of the full beams, with the least bias on AGBD.
# Cover beams 1 - 4 underestimate. Full beams 7 - 8 overestimate. 5 - 6 tend give average values.

# Check for GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0': print('GPU device not found')
else: print(f"Found GPU at: {device_name}")

# Load model

# Reload hyperparameters
with open(selected_model_descr_dir) as model_description_json:
  model_description = json.load(model_description_json)
final_hyperparameters = eval(model_description["hyperparameters"])

# Remove early stopping and replace with mean n_estimators
if "early_stopping_rounds" in final_hyperparameters:
  final_hyperparameters = {k:v for k, v in final_hyperparameters.items() if k != "early_stopping_rounds"}
  final_hyperparameters["n_estimators"] = round(model_description["n_estimators mean"])

if categorise_variate: XGBPredictor = xgb.XGBClassifier(**final_hyperparameters)
else: XGBPredictor = xgb.XGBRegressor(**final_hyperparameters)
XGBPredictor.load_model(fname=selected_model_json)

# Avoids issues using dataframe from CPU
xgb.set_config(verbosity=0, use_rmm=True)

# Check existing tile parameters
template_tile_list = []
for file in os.listdir(tile_templates_dir):
  if file.endswith('.tif') and file[:13] == 'template_tile':
    template_tile_list.append(file)
n_tiles = len(template_tile_list)

assert n_tiles > 0, "There are no template tiles. Run the template tiles section, even if only one is created."
template_tile_x = gdal.Open(join(tile_templates_dir,'template_tile_1.tif')).GetRasterBand(1).XSize
print(f"There are {n_tiles} template tiles.")

# Scenario progress
scenario_progress_index = 0
scenario_progress_label = widgets.Label(f"Scenario progress: {scenario_progress_index}/{len(scenarios_to_predict)}")
display(scenario_progress_label)

# Tile progress
tile_progress_index, tile_progress_label = 0, widgets.Label(value=f"Tile progress: 0 / {n_tiles}")
display(tile_progress_label)

# Loop through each scenario
for scenario in scenarios_to_predict:
  # Define scenario filename and check if exists
  scenario_prediction_unmasked_filename = f"{scenario}__{selected_scenario_area}_{selected_model}_unmasked.tif"
  scenario_prediction_unmasked_exists=False
  for scenario_prediction in os.listdir(scenario_predictions_unmasked_dir):
    if scenario_prediction == scenario_prediction_unmasked_filename:
      scenario_prediction_unmasked_exists=True
  # If scenario prediction does not exist:
  if scenario_prediction_unmasked_exists == False:
    # Get number of stacks
    scenario_predictor_stack_dir = join(tile_predictor_stacks_dir, scenario)
    n_stacks = len(os.listdir(scenario_predictor_stack_dir))
    # Create a tile cache directory for the prediction
    tile_cache_scenario_dir = join(tile_prediction_cache_dir, scenario_prediction_unmasked_filename[:-4])
    makedirs(tile_cache_scenario_dir, exist_ok=True)
    # Create a tile count to match the predictor stack chunk
    for stack in range(1, n_stacks+1):
      scenario_tile_filename = f"scenario_tile_{stack}.tif"
      # Check if tile already exists
      scenario_tile_exists = False
      for scenario_tile in os.listdir(tile_cache_scenario_dir):
        if scenario_tile == scenario_tile_filename: scenario_tile_exists=True
      # If scenario prediction tile does not exist:
      if scenario_tile_exists == False:
        # Load template tile parameters
        template_tile_dir = join(tile_templates_dir, f"template_tile_{stack}.tif")
        template_tile_y = gdal.Open(template_tile_dir).GetRasterBand(1).YSize
        template_tile_x = gdal.Open(template_tile_dir).GetRasterBand(1).XSize
        # Load predictor tile stack
        stack_filename = f"predictor_stack_{scenario}_{stack}.npy"
        predictor_stack = np.load(join(scenario_predictor_stack_dir, stack_filename))
        # Add covariates (sensitivity and BEAM)
        if add_covariates: predictor_stack = np.hstack((predictor_stack,
                           np.full((predictor_stack.shape[0], 1), beam_value, dtype=int),
                           np.full((predictor_stack.shape[0], 1), sensitivity_value, dtype=float)
                           ))
        # Define prediction array and reshape
        prediction = XGBPredictor.predict(predictor_stack)
        predictor_stack = None # Flush predictor stack
        prediction_tile = prediction.reshape((template_tile_y, template_tile_x))
        prediction = None # Flush prediction
        # Export prediction array as .tif
        export_array_as_tif(prediction_tile, join(tile_cache_scenario_dir, scenario_tile_filename), template = template_tile_dir, compress = False)
        prediction_tile = None # Flush prediction tile
        # Update progress
      tile_progress_index += 1
      tile_progress_label.value = f"Tile progress: {tile_progress_index} / {n_stacks}"
    # Prepare empty array for merging tiles
    prediction_array = np.empty((0,template_tile_x))
    # Read each tile .tif as an array, stack, then export as a .tif
    for subdir in os.listdir(tile_cache_scenario_dir):
      if subdir.endswith('.tif'):
        tile_dir = join(tile_cache_scenario_dir, subdir)
        prediction_array = np.vstack((prediction_array, gdal.Open(tile_dir).ReadAsArray()))
    # Define scenario template
    scenario_template = join(predictors_dir, os.listdir(predictors_dir)[0])
    scenario_prediction_unmasked_dir = join(scenario_predictions_unmasked_dir, f"{scenario_prediction_unmasked_filename}")
    export_array_as_tif(prediction_array, scenario_prediction_unmasked_dir, template = scenario_template, compress = True)
    # Delete scenario tile cache directory
    shutil.rmtree(tile_cache_scenario_dir)
  # Reset tile progress
  tile_progress_index = 0
  # Update scenario progress
  scenario_progress_index += 1
  scenario_progress_label.value = f"Scenario progress: {scenario_progress_index}/{len(scenarios_to_predict)}"
print("\nScenario predictions complete.")

# Mask scenario predictions



In [ ]:
# Use polygons for masking, only areas inside the polygons will be included

# Exclude existing polygons from search
polygons_to_exclude = ['template.gpkg', 'project_area_buffered_bbox.gpkg']

print("mask_polygons = [")
for polygon in os.listdir(polygons_dir):
  if polygon not in polygons_to_exclude:
    if 'inverse' not in polygon:
      print(f"  '{polygon[:-5]}',")
print("]")

In [ ]:
mask_polygons = [
  # 'project_area',
  'gedi_area',
  # 'peninsular_malaysia',
  # 'pa_taman_krau',
  # 'pa_ais',
]

# Create an inverse project area path for masking
template_polygon_path = join(polygons_dir, "template.gpkg")
for polygon in mask_polygons:
  inverse_polygon_path = join(polygons_dir, f"{polygon}_inverse.gpkg")
  if not exists(inverse_polygon_path):
    polygon_path = join(polygons_dir, f"{polygon}.gpkg")
    template_polygon = gpd.read_file(template_polygon_path)
    polygon_read = gpd.read_file(polygon_path)
    polygon_crs = polygon_read.crs.to_epsg()
    inverse_polygon = template_polygon['geometry'].difference(polygon_read['geometry']).iloc[0]
    inverse_polygon_gdf = gpd.GeoDataFrame({'geometry': [inverse_polygon]}, crs=f"EPSG:{polygon_crs}")
    inverse_polygon_gdf.to_file(inverse_polygon_path, driver="GPKG")
    print(f"An inverse masking polygon for {polygon} has been created in {polygons_dir}.")
  else: print(f"An inverse masking polygon for {polygon} already exists.")

unmasked_predictions = []
for scenario_prediction in os.listdir(scenario_predictions_unmasked_dir):
  unmasked_predictions.append(scenario_prediction)

# Determine last predictor year for masking future scenarios
final_predictor_years = []
for final_predictor in os.listdir(predictor_final_dir):
  if final_predictor.endswith('.tif') and final_predictor[-9] == '_':
    try: final_predictor_years.append(int(final_predictor[-8:-4]))
    except: continue
last_predictor_year = max(final_predictor_years)

# Binary progress
masking_progress_index = 0
masking_progress_label = widgets.Label(f"Masking progress: {masking_progress_index}/{len(unmasked_predictions)}")
display(masking_progress_label)

# Mask scenarios with the relevatant mask
for scenario_prediction in unmasked_predictions: # Loop through each unmasked scenario
  if 'nodeg_nodef' in scenario_prediction: scenario_year = int(scenario_prediction.split('__')[0][-4:])
  else: scenario_year = int(scenario_prediction[:4])
  scenario_masked_filename = f"{scenario_prediction[:-13]}.tif"
  scenario_masked_dir = join(scenario_predictions_dir, scenario_masked_filename)
  if not exists(scenario_masked_dir):
    mask_exists = False
    for mask in os.listdir(masks_dir):
      mask_year = int(mask[12:16])
      # Match oldgrowth all land scenarios
      if 'all_land' in mask:
        if 'all_land' in scenario_prediction:
          selected_mask_filename = mask
          selected_mask_dir = join(masks_dir, selected_mask_filename)
          mask_exists = True
      else: # Match all other historic scenarios
        if scenario_year == mask_year:
          selected_mask_filename = mask
          selected_mask_dir = join(masks_dir, selected_mask_filename)
          mask_exists = True
        else: # Match future scenarios with most recent forest mask
          if scenario_year > last_predictor_year and last_predictor_year == mask_year:
            selected_mask_filename = mask
            selected_mask_dir = join(masks_dir, selected_mask_filename)
            mask_exists = True
    if mask_exists == False: print(f"A suitable mask for {scenario_prediction} does not exist.\n")
    else: # Mask the scenario prediction
      print(f"Masking {scenario_prediction} with {selected_mask_filename}...")
      mask_array = gdal.Open(selected_mask_dir).ReadAsArray()
      scenario_prediction_unmasked_dir = join(scenario_predictions_unmasked_dir, scenario_prediction)
      scenario_prediction_array = gdal.Open(scenario_prediction_unmasked_dir).ReadAsArray()
      # Mask where the mask array is not 1
      scenario_masked_array = np.where(mask_array != 1, nodatavalue, scenario_prediction_array)
      export_array_as_tif(scenario_masked_array, scenario_masked_dir, compress = True)
      if len(mask_polygons) > 0:
        for polygon_mask in mask_polygons:
          inverse_gedi_area_path = join(polygons_dir, f"{polygon_mask}_inverse.gpkg")
          print(f"Masking {scenario_prediction} with {polygon_mask}...")
          burn_polygon_to_raster(scenario_masked_dir, inverse_gedi_area_path, fixed_value=nodatavalue, all_touched=False)
        # Recompress the prediction after burning the polygon masks
        scenario_masked_array_2 = gdal.Open(scenario_masked_dir).ReadAsArray()
        export_array_as_tif(scenario_masked_array_2, scenario_masked_dir, compress = True)
      print(f"{scenario_masked_filename} exported.")
  # Update masking progress
  masking_progress_index += 1
  masking_progress_label.value = f"Masking progress: {masking_progress_index}/{len(unmasked_predictions)}"

# Scenario difference

In [ ]:
scenarios_diff_set = set()
for masked_scenario in os.listdir(scenario_predictions_dir):
    scenarios_diff_set.add(masked_scenario.split("__")[0])

# Generate all possible pairs of scenarios, including all orders
scenario_pairs = sorted(list(itertools.permutations(scenarios_diff_set, 2)))

print("# Select scenarios to calculate mean difference with uncertainty")
print("scenario_pairs = [")
for s1, s2 in scenario_pairs:
    print(f" ('{s1}','{s2}'),")
print("]")

In [ ]:
# Select scenarios to calculate mean difference with uncertainty
scenario_pairs = [
 ('1990_oldgrowth','1990_oldgrowth_all_land'),
 ('2023','1990_oldgrowth_all_land'),
 ('2023','2023_nodeg_nodef_since_1990'),
 ('2023','2023_nodeg_since_1990'),
 ('2023','2023_oldgrowth'),
 ('2023_nodeg_nodef_since_1990','1990_oldgrowth_all_land'),
 ('2023_nodeg_since_1990','2023_oldgrowth'),
 ('2023_oldgrowth','1990_oldgrowth'),
 ('2023_oldgrowth','1990_oldgrowth_all_land'),
]

In [ ]:
# Rename scenario differences for semantic meaning (optional)
difference_names = {
    ('1990_oldgrowth', '1990_oldgrowth_all_land'): 'deforestation_before_1990',
    ('2023', '1990_oldgrowth_all_land'): 'degradation_deforestation_total',
    ('2023', '2023_nodeg_nodef_since_1990'): 'degradation_deforestation_since_1990',
    ('2023', '2023_nodeg_since_1990'): 'degradation_since_1990',
    ('2023', '2023_oldgrowth'): 'degradation_total',
    ('2023_nodeg_nodef_since_1990', '1990_oldgrowth_all_land'): 'degradation_deforestation_before_1990',
    ('2023_nodeg_since_1990', '2023_oldgrowth'): 'degradation_before_1990',
    ('2023_oldgrowth', '1990_oldgrowth'): 'deforestation_since_1990',
    ('2023_oldgrowth', '1990_oldgrowth_all_land'): 'deforestation_total'
}

In [ ]:
# Function for difference
def diff(scenario1, scenario2):
  diff_array = scenario1 - scenario2
  return diff_array

# Loop through the scenario pairs
for scenario1, scenario2 in scenario_pairs:

  # Lookup the description from the dictionary
  difference_name = difference_names.get((scenario1, scenario2), f"{scenario1}_-_{scenario2}")

  # Define filenames and directories of difference .tifs
  diff_filename = f"{difference_name}__{selected_scenario_area}_{selected_model}.tif"
  diff_dir = join(scenario_diff_dir, diff_filename)

  if not exists(diff_dir):
    print(f"Calculating difference between {scenario1} and {scenario2}")
    scenario1_filename = f"{scenario1}__{selected_scenario_area}_{selected_model}.tif"
    scenario2_filename = f"{scenario2}__{selected_scenario_area}_{selected_model}.tif"

    # Define difference directories, assert that both exist for both scenarios
    scenario1_dir = join(scenario_predictions_dir, scenario1_filename)
    assert exists(scenario1_dir), f"{scenario1_dir} does not exist."
    scenario2_dir = join(scenario_predictions_dir, scenario2_filename)
    assert exists(scenario2_dir), f"{scenario1_dir} does not exist."

    # Convert scenario .tifs to temporary arrays
    scenario1_array_temp = gdal.Open(scenario1_dir).ReadAsArray()
    scenario2_array_temp = gdal.Open(scenario2_dir).ReadAsArray()

    # Fill scenario nodata values with 0 if they are not nodatavalues in the other scenario
    scenario1_array = np.where((scenario1_array_temp == nodatavalue) & (scenario2_array_temp != nodatavalue), 0, scenario1_array_temp)
    scenario2_array = np.where((scenario2_array_temp == nodatavalue) & (scenario1_array != nodatavalue), 0, scenario2_array_temp)

    # Create difference arrays where the value is not 'nodatavalue'
    diff_array = np.where(scenario1_array==nodatavalue, nodatavalue, diff(scenario1_array, scenario2_array))
    export_array_as_tif(diff_array, diff_dir, template = scenario1_dir), print(f"{diff_filename} has been exported.")

  else: print(f"{diff_filename} already exists.")

# Disconnect runtime

In [ ]:
# Useful for stopping background execution
runtime.unassign()